# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.5247e19,"""5179392153921583691""","""../../../../data/SampleHDS.jso…","""17474175563586727886""","""9714192072175555228""","""6099134761902327241""",false
1.8460e18,"""17940013979327734757""","""../../../../data/SampleHDS.jso…","""14981675891994343769""","""9714192072175555228""","""18294008364412038654""",true
1.1197e19,"""14999552791562091104""","""../../../../data/SampleHDS.jso…","""8262673382741712242""","""12645135992074137864""","""6099134761902327241""",false
2.7843e18,"""17940013979327734757""","""../../../../data/SampleHDS.jso…","""2907777176371979799""","""11230190041859322853""","""6099134761902327241""",false
7.6730e18,"""5179392153921583691""","""../../../../data/SampleHDS.jso…","""7497014876548220268""","""11230190041859322853""","""9469689233780073754""",true
6.6172e18,"""14999552791562091104""","""../../../../data/SampleHDS.jso…","""14205187435825479763""","""11445304671692733038""",null,false
2.7919e18,"""14999552791562091104""","""../../../../data/SampleHDS.jso…","""14541675697494601701""","""11230190041859322853""","""9469689233780073754""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
5.4735e17,"""15382727669290606416""","""../../../../data/SampleHDS.jso…","""8063140318070100244""","""FirstMortgage30yr""","""12730265499883026579""","""Rejected"""
1.4114e19,"""11890983927289252095""","""../../../../data/SampleHDS.jso…","""12107142067298034732""","""FirstMortgage30yr""","""4169497361815982117""","""Accepted"""
1.4733e19,"""7602069423440632356""","""../../../../data/SampleHDS.jso…","""5274731381479035942""","""MoneyMarketSavingsAccount""","""12730265499883026579""","""Rejected"""
1.0467e19,"""11890983927289252095""","""../../../../data/SampleHDS.jso…","""17197415458613310218""","""BasicChecking""","""12730265499883026579""","""Rejected"""
3.0848e18,"""15382727669290606416""","""../../../../data/SampleHDS.jso…","""16907010810793496991""","""BasicChecking""","""8479992277171098576""","""Accepted"""
1.6352e19,"""7602069423440632356""","""../../../../data/SampleHDS.jso…","""17095475831976436801""","""UPlusFinPersonal""",null,"""Rejected"""
4.8860e18,"""7602069423440632356""","""../../../../data/SampleHDS.jso…","""15191262926429173632""","""BasicChecking""","""8479992277171098576""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.6016e19,"""1036680771002603838""","""../../../../data/SampleHDS.jso…","""2462089141645756336""","""FirstMortgage30yr""","""38513706541725791""","""Rejected"""
1.8127e19,"""14044617397041215286""","""../../../../data/SampleHDS.jso…","""10809778988135924044""","""FirstMortgage30yr""","""13746767746219736417""","""Accepted"""
1.3075e19,"""2411782643901597225""","""../../../../data/SampleHDS.jso…","""13443638259097896003""","""MoneyMarketSavingsAccount""","""38513706541725791""","""Rejected"""
6.8473e18,"""14044617397041215286""","""../../../../data/SampleHDS.jso…","""8360695290463518531""","""BasicChecking""","""38513706541725791""","""Rejected"""
9.8768e18,"""1036680771002603838""","""../../../../data/SampleHDS.jso…","""18238277421133152802""","""BasicChecking""","""5115345972055596829""","""Accepted"""
7.9551e18,"""2411782643901597225""","""../../../../data/SampleHDS.jso…","""154043692009781937""","""UPlusFinPersonal""",null,"""Rejected"""
1.0464e19,"""2411782643901597225""","""../../../../data/SampleHDS.jso…","""15100455684642642473""","""BasicChecking""","""5115345972055596829""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
filename=filename
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.5967e19,"""2694305331326908706""","""1840724640545968672""","""FirstMortgage30yr""","""4939638654962732341""","""Rejected"""
1.5737e19,"""1949180108809398051""","""7832152552962031928""","""FirstMortgage30yr""","""12826971976887187280""","""Accepted"""
1.4409e19,"""16884734000232782142""","""15412287632562621725""","""MoneyMarketSavingsAccount""","""4939638654962732341""","""Rejected"""
1.4422e19,"""1949180108809398051""","""17733056944395838154""","""BasicChecking""","""4939638654962732341""","""Rejected"""
1.2182e19,"""2694305331326908706""","""917926020296609547""","""BasicChecking""","""16267193307825368447""","""Accepted"""
1.1628e19,"""16884734000232782142""","""15498909199646823918""","""UPlusFinPersonal""",null,"""Rejected"""
7.5535e17,"""16884734000232782142""","""5931328156118746873""","""BasicChecking""","""16267193307825368447""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'